# Исследование данных сервиса “Яндекс.Музыка”

Сравнение поведения пользователей "Яндекс Музыки" в Москве и Санкт-Петербурге

# Цель проекта

Проверить данные и сравнить поведение и предпочтения пользователей из Москвы и Санкт-Петербурга.

Выявить закономерности.

# Структура проекта

1. Загрузка и изучение общей информации о данных
2. Предобработка данных
3. Проверка гипотез
4. Итоги исследования

# Загрузка и изучение общей информации о данных

## Импорт библиотек

In [1]:
import pandas as pd

## Открытие файла

In [2]:
try:
    df = pd.read_csv('/datasets/yandex_music_project.csv')
except:
    df = pd.read_csv('https://code.s3.yandex.net/datasets/yandex_music_project.csv')

## Просмотр общей информации

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63848 non-null  object
 2   artist    57876 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


Итак, в таблице семь столбцов. Тип данных во всех столбцах — object.

Согласно документации к данным:
- userID — идентификатор пользователя;
- Track — название трека;
- artist — имя исполнителя;
- genre — название жанра;
- City — город пользователя;
- time — время начала прослушивания;
- Day — день недели.

Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.

# Предобработка данных

## Приведу названия столбцов к нижнему регистру

In [5]:
df = df.rename(columns={"  userID" : "user_id" , "Track" : "track" , "  City  " : "city" , "Day" : "day"})

## Проверка на пропуски в столбцах

In [7]:
df.isna().sum()

user_id       0
track      1231
artist     7203
genre      1198
city          0
time          0
day           0
dtype: int64

Пропуски в столбцах track и artist не важны для исследования - заменю их явным обозначением.

Пропуски в столбце genre могут помешать исследованию, но возможности восстановить данные нет. Также заменю их явным обозначением

## Заменю пропущенные значения в столбцах track, artist и genre на 'unknown

In [8]:
columns_to_replace = ["track" , "artist" , "genre"]
for column in columns_to_replace:
    df[column] = df[column].fillna("unknown")

## Проверка на дубликаты

In [10]:
df.duplicated().sum()

3826

## Удаление явных дубликатов

In [11]:
df = df.drop_duplicates()

## Удаление неявных дубликатов в столбце genre

### Просмотр уникальных жанров

In [13]:
genre = df["genre"].sort_values()
genre.unique()

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

### Заменю жанры hip, hop и hip-hop на hiphop

In [14]:
df = df.replace("hip" , "hiphop")
df = df.replace("hop" , "hiphop")
df = df.replace("hip-hop" , "hiphop")

Предобработка обнаружила три проблемы в данных:

- нарушения в стиле заголовков,
- пропущенные значения,
- дубликаты — явные и неявные.

Исправили заголовки, чтобы упростить работу с таблицей. Без дубликатов исследование станет более точным.

Пропущенные значения заменил на unknown.

# Проверка гипотез

## Сравнение поведения пользователей двух столиц

Проверю предположение, что пользователи по-разному слушают музыку в Москве и Санкт-Петербурге. Проверю это предположение по данным о трёх днях недели — понедельнике, среде и пятнице.

* Разделю пользователей Москвы и Санкт-Петербурга.
* Сравню, сколько треков послушала каждая группа пользователей в понедельник, среду и пятницу.

### Подсчет прослушиваний в каждом городе

In [16]:
col = df.groupby("city")["time"].count()
col

city
Moscow              42741
Saint-Petersburg    18512
Name: time, dtype: int64

В Москве прослушиваний больше, чем в Петербурге. Из этого не следует, что московские пользователи чаще слушают музыку. Просто самих пользователей в Москве больше.

### Подсчет прослушиваний в каждый из трёх дней

In [17]:
col = df.groupby("day")["time"].count()
col

day
Friday       21840
Monday       21354
Wednesday    18059
Name: time, dtype: int64

В среднем пользователи из двух городов менее активны по средам. Но картина может измениться, если рассмотреть каждый город в отдельности.

### Напишу функцию для подсчета прослушиваний для конкретного города и дня

In [18]:
def number_tracks(day , city):
    track_list = df[df["day"] == day]
    track_list = track_list[track_list["city"] == city]
    track_list_count = track_list["user_id"].count()
    return track_list_count

### Рассматрю количество прослушиваний в Москве и Санкт-Петербурге по каждому дню

In [39]:
print('Прослушиваний в Москве по понедельникам:',number_tracks("Monday" , "Moscow"))

Прослушиваний в Москве по понедельникам: 15740


In [41]:
print('Прослушиваний в Санкт-Петербурге по понедельникам:',number_tracks("Monday" , "Saint-Petersburg"))

Прослушиваний в Санкт-Петербурге по понедельникам: 5614


In [43]:
print('Прослушиваний в Москве по средам:',number_tracks("Wednesday" , "Moscow"))

Прослушиваний в Москве по средам: 11056


In [44]:
print('Прослушиваний в Санкт-Петербурге по средам:',number_tracks("Wednesday" , "Saint-Petersburg"))

Прослушиваний в Санкт-Петербурге по средам: 7003


In [45]:
print('Прослушиваний в Москве по пятницам:',number_tracks("Friday" , "Moscow"))

Прослушиваний в Москве по пятницам: 15945


In [46]:
print('Прослушиваний в Санкт-Петербурге по пятницам:',number_tracks("Friday" , "Saint-Petersburg"))

Прослушиваний в Санкт-Петербурге по пятницам: 5895


### Строю таблицу с результатами

In [25]:
pd.DataFrame(data = [["Moscow" , 15740 , 11056 , 15945] , ["Saint-Petersburg" , 5614 , 7003 , 5895]] , columns = ["city" , "monday" , "wednesday" , "Friday"])

,city,monday,wednesday,Friday
0,Moscow,15740,11056,15945
1,Saint-Petersburg,5614,7003,5895


Данные показывают разницу поведения пользователей:

- В Москве пик прослушиваний приходится на понедельник и пятницу, а в среду заметен спад.
- В Петербурге, наоборот, больше слушают музыку по средам. Активность в понедельник и пятницу здесь почти в равной мере уступает среде.

Значит, данные говорят о существующей разнице в прослушиваниях.

## Музыка в начале и в конце недели

Согласно второй гипотезе, утром в понедельник в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города.

### Разделю данные по городам

In [26]:
moscow_general = df[df["city"] == "Moscow"]

In [27]:
spb_general = df[df["city"] == "Saint-Petersburg"]

### Создам функцию для нахождения топ-10 жанров тех треков, которые прослушивали в указанный день, в промежутке между двумя отметками времени

Функция будет принимать четыре параметра:
- таблица с данными,
- день недели,
- начальная временная метка в формате,
- последняя временная метка в формате.

In [47]:
def genre_weekday(df, day, time1, time2):
    # последовательная фильтрация
    # оставляю в genre_df только те строки df, у которых день равен day
    genre_df = df[df["day"] == day]
    # оставляю в genre_df только те строки genre_df, у которых время меньше time2
    genre_df = genre_df[genre_df["time"] > time1]
    # оставляю в genre_df только те строки genre_df, у которых время больше time1
    genre_df = genre_df[genre_df["time"] < time2]
    # сгруппирую отфильтрованный датафрейм по столбцу с названиями жанров, возьму столбец genre и посчитаю кол-во строк для каждого жанра методом count()
    genre_df_grouped = genre_df.groupby("genre")["genre"].count()
    # отсортирую результат по убыванию (чтобы в начале Series оказались самые популярные жанры)
    genre_df_sorted = genre_df_grouped.sort_values(ascending=False)
    # верну Series с 10 самыми популярными жанрами в указанный отрезок времени заданного дня
    return genre_df_sorted[:10]

### Cравню результаты функции для Москвы и Санкт-Петербурга в понедельник утром (с 7:00 до 11:00) и в пятницу вечером (с 17:00 до 23:00)

In [50]:
genre_weekday(moscow_general , "Monday" , "07:00" , "11:00")

genre
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
unknown        161
Name: genre, dtype: int64

In [30]:
genre_weekday(spb_general , "Monday" , "07:00" , "11:00")

genre
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
Name: genre, dtype: int64

In [31]:
genre_weekday(moscow_general , "Friday" , "17:00" , "23:00")

genre
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
alternative    163
classical      163
rusrap         142
Name: genre, dtype: int64

In [32]:
genre_weekday(spb_general , "Friday" , "17:00" , "23:00")

genre
pop            256
electronic     216
rock           216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap          59
world           54
Name: genre, dtype: int64

Если сравнить топ-10 жанров в понедельник утром, можно сделать такие выводы:

В Москве и Петербурге слушают похожую музыку. Единственное отличие — в московский рейтинг вошёл жанр “world”, а в петербургский — джаз и классика.

В Москве пропущенных значений оказалось так много, что значение 'unknown' заняло десятое место среди самых популярных жанров. Значит, пропущенные значения занимают существенную долю в данных и угрожают достоверности исследования.

Вечер пятницы не меняет эту картину. Некоторые жанры поднимаются немного выше, другие спускаются, но в целом топ-10 остаётся тем же самым.

Таким образом, вторая гипотеза подтвердилась лишь частично:
Пользователи слушают похожую музыку в начале недели и в конце.

Разница между Москвой и Петербургом не слишком выражена. В Москве чаще слушают русскую популярную музыку, в Петербурге — джаз.

Однако пропуски в данных ставят под сомнение этот результат. В Москве их так много, что рейтинг топ-10 мог бы выглядеть иначе, если бы не утерянные данные о жанрах.

## Жанровые предпочтения в Москве и Петербурге

Гипотеза: Петербург — столица рэпа, музыку этого жанра там слушают чаще, чем в Москве. А Москва — город контрастов, в котором, тем не менее, преобладает поп-музыка.

### Сгруппирую таблицу moscow_general по жанру и посчитаю прослушивания треков каждого жанра

In [33]:
moscow_general = df[df["city"] == "Moscow"]
moscow_genres = moscow_general.groupby("genre")["genre"].count()
moscow_genres = moscow_genres.sort_values(ascending=False)

In [34]:
print(moscow_genres.head(10))

genre
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: genre, dtype: int64


### Сгруппирую таблицу spb_general по жанру и посчитаю прослушивания треков каждого жанра

In [35]:
spb_general = df[df["city"] == "Saint-Petersburg"]
spb_genres = spb_general.groupby("genre")["genre"].count()
spb_genres = spb_genres.sort_values(ascending=False)

In [36]:
print(spb_genres.head(10))

genre
pop            2431
dance          1932
rock           1879
electronic     1736
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: genre, dtype: int64


Гипотеза частично подтвердилась:

Поп-музыка — самый популярный жанр в Москве, как и предполагала гипотеза. Более того, в топ-10 жанров встречается близкий жанр — русская популярная музыка.

Вопреки ожиданиям, рэп одинаково популярен в Москве и Петербурге.

# Итоги исследования 

Проверил три гипотезы и установил:

1. День недели по-разному влияет на активность пользователей в Москве и Петербурге.
Первая гипотеза полностью подтвердилась.

2. Музыкальные предпочтения не сильно меняются в течение недели — будь то Москва или Петербург. Небольшие различия заметны в начале недели, по понедельникам:
- в Москве слушают музыку жанра “world”,
- в Петербурге — джаз и классику.

Таким образом, вторая гипотеза подтвердилась лишь отчасти. Этот результат мог оказаться иным, если бы не пропуски в данных.
3. Во вкусах пользователей Москвы и Петербурга больше общего чем различий. Вопреки ожиданиям, предпочтения жанров в Петербурге напоминают московские.

Третья гипотеза не подтвердилась. Если различия в предпочтениях и существуют, на основной массе пользователей они незаметны.